In [ ]:
from os import chdir 
chdir('./lib')

In [2]:
from project_5 import general_process, load_data_from_database, make_data_dict, general_model, general_transformer, general_modelGS

# Step 3 - Build Model

### Domain and Data

MADELON is an artificial dataset, which was part of the NIPS 2003 feature selection challenge. This is a two-class classification problem with continuous input variables. The difficulty is that the problem is multivariate and highly non-linear.
MADELON is an artificial dataset containing data points grouped in 32 clusters placed on the vertices of a five dimensional hypercube and randomly labeled +1 or -1. The five dimensions constitute 5 informative features. 15 linear combinations of those features were added to form a set of 20 (redundant) informative features. Based on those 20 features one must separate the examples into the 2 classes (corresponding to the +-1 labels). A number of distractor feature called 'probes' were added having no predictive power. The order of the features and patterns were randomized.

### Problem Statement

We are going to further filter our features using a simple Grid Search Cross Validation model.

### Solution Statement

In order to do this, we are going to run a Grid Search Cross Validation through our data
We will construct a Pipeline that uses SelectKBest to transform data
We will construct a Pipeline that uses LogisticRegression to model data
We will construct a Pipeline that uses KNearestNeighbors to model data
And Gridsearch optimal parameters for logistic regression and KNN

### Metric

Our metric is our accuracy score on our feature selection

### Benchmark

Our benchmark hasn't changed since step 1.

## Implementation

Implement the following code pipeline using the functions you write in `lib/project_5.py`.

<img src="assets/build_model.png" width="600px">

In [3]:
# Let's use the function we wrote to load our data into the dataframe
df = load_data_from_database()
df.shape

(2000, 502)

In [4]:
# Let's get info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Columns: 502 entries, index to label
dtypes: int64(502)
memory usage: 7.7 MB


In [5]:
# Describe the Data
df.describe()

,index,feat_000,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_491,feat_492,feat_493,feat_494,feat_495,feat_496,feat_497,feat_498,feat_499,label
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.00000
mean,999.500000,481.722500,483.452500,510.166000,483.384500,501.612500,479.259000,480.109500,476.565000,486.793500,...,478.811500,486.356500,496.565500,493.49950,510.893000,478.219500,483.309000,507.977000,490.266000,0.00000
std,577.494589,6.421769,30.186294,38.899165,9.059895,41.389418,6.795956,40.575925,1.384461,15.043836,...,4.011735,23.967366,127.635442,34.81902,37.459353,5.880613,13.559847,37.224297,25.825273,1.00025
min,0.000000,462.000000,381.000000,370.000000,453.000000,371.000000,459.000000,334.000000,471.000000,430.000000,...,463.000000,391.000000,130.000000,368.00000,398.000000,457.000000,435.000000,363.000000,403.000000,-1.00000
25%,499.750000,477.000000,464.000000,485.000000,477.000000,475.000000,475.000000,452.750000,476.000000,477.000000,...,476.000000,471.000000,404.000000,470.00000,486.000000,474.000000,474.000000,482.000000,473.000000,-1.00000
50%,999.500000,482.000000,483.000000,510.500000,483.000000,500.000000,479.000000,480.000000,477.000000,487.000000,...,479.000000,486.000000,504.000000,492.00000,511.000000,478.000000,483.000000,508.000000,490.000000,0.00000
75%,1499.250000,486.000000,503.000000,536.000000,490.000000,528.000000,484.000000,506.250000,477.000000,496.250000,...,481.000000,502.000000,586.000000,517.00000,535.000000,482.000000,492.000000,533.000000,507.250000,1.00000
max,1999.000000,503.000000,600.000000,654.000000,519.000000,688.000000,505.000000,611.000000,481.000000,536.000000,...,497.000000,566.000000,920.000000,615.00000,661.000000,500.000000,535.000000,644.000000,583.000000,1.00000


In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.datasets import samples_generator
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import make_pipeline


In [22]:
# Let's load the data , run a Standard scaler and SelectKBest transformer through a 
# Grid Search with a Logistic regression...our own little pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

gs_params = {
              'C': [.01, .02, .03, .05, .06, .07, .08, .09, .1]
}

this_dd = make_data_dict(df, random_state=0)
this_scale = general_transformer(StandardScaler(), this_dd)
this_kbest = general_transformer(SelectKBest(k=20), this_scale)
gs = GridSearchCV(LogisticRegression(penalty='l1'), param_grid=gs_params)
this_gs = general_modelGS(gs, this_dd)
this_gs

{'X_test': array([[-2.0060363 ,  0.96262702, -2.25549538, ...,  1.52381869,
         -1.48860853, -0.81983392],
        [-1.37226453, -1.87040739, -0.33279748, ...,  1.38890779,
          1.34491786, -0.07652201],
        [ 0.43515865, -0.52667565,  0.63968679, ...,  1.16405627,
         -0.02484371,  1.23796644],
        ..., 
        [-0.31597826,  0.59310079, -0.3699152 , ...,  1.88358111,
         -1.71690213,  2.50550886],
        [ 1.84354035,  1.10819796, -0.23629141, ..., -0.2749934 ,
         -0.60229222, -1.15628037],
        [-0.69154671,  1.48892195, -1.6170706 , ..., -1.93889458,
          0.49888865, -1.79005158]]),
 'X_train': array([[ 1.06893041, -0.19074272,  0.78815767, ...,  1.63624445,
         -0.96487617,  0.82327663],
        [-0.104721  ,  0.61549632, -0.54065671, ...,  1.79364051,
         -1.44832143,  0.91716866],
        [-0.92627699, -1.20973928,  0.06065035, ...,  0.98417507,
          0.49888865,  0.07214038],
        ..., 
        [-0.19861312,  0.078003

In [23]:
this_gs['train_score'], this_gs['test_score']

(0.6226666666666667, 0.61199999999999999)

In [15]:
results = this_kbest['transform'].get_support()

In [16]:
import numpy as np
np.where(results)

(array([ 49,  56,  65, 106, 129, 162, 200, 205, 242, 283, 297, 337, 339,
        348, 379, 443, 454, 473, 476, 494]),)

In [24]:
# Let's load the data , run a Standard scaler and SelectKBest transformer through a 
# Grid Search with a Logistic regression...our own little pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

gs_params = {
              'n_neighbors': [5, 7, 9, 11, 13, 15, 17, 19, 21]
}

this_dd = make_data_dict(df, random_state=0)
this_scale = general_transformer(StandardScaler(), this_dd)
this_kbest = general_transformer(SelectKBest(k=20), this_scale)
gs = GridSearchCV(KNeighborsClassifier(), param_grid=gs_params)
this_gs = general_modelGS(gs, this_dd)
this_gs

{'X_test': array([[-1.14661665, -0.71195696, -1.33243707, ...,  0.14735686,
         -0.01614779, -0.38668972],
        [-1.44914174, -0.9046903 ,  1.8322541 , ..., -1.10592215,
         -0.80052878,  0.64174038],
        [-0.40193949, -0.83241529, -1.21900728, ...,  0.55164041,
         -0.43971353,  0.29893035],
        ..., 
        [-1.12334548, -4.06069874, -0.47037066, ...,  1.26587469,
         -1.67119169, -0.55809473],
        [-0.84409155,  0.32398474,  0.50512555, ...,  1.05025679,
         -1.50647168, -0.04387968],
        [ 0.48236463,  0.97445977,  0.42572469, ..., -0.3377834 ,
          0.2113227 , -0.55809473]]),
 'X_train': array([[ 0.41255114,  0.27580141,  0.50512555, ..., -0.35125952,
         -0.07105446,  0.29893035],
        [ 0.11002605,  0.61308475,  1.37853494, ..., -1.01158932,
         -0.03183541, -1.2437148 ],
        [-0.56483762,  1.62493479, -0.15276724, ..., -0.7016386 ,
          0.64273224, -1.58652484],
        ..., 
        [-0.96044736,  1.167193

In [25]:
this_gs['train_score'], this_gs['test_score']

(0.84466666666666668, 0.78400000000000003)

In [28]:
# And here are our most important features
results = this_kbest['transform'].get_support()
knnresults = np.where(results)
knnresults

(array([ 49,  57,  65, 106, 120, 129, 200, 206, 242, 286, 337, 339, 379,
        385, 443, 454, 473, 476, 494, 497]),)

# Step 3 Conclusion:

We constructed a Pipeline that uses StandardScaler and SelectKBest to transform the data, a Grid Search with an embedded LogisticRegression to model. Finally, we used Gridsearch optimal parameters for logistic regression and KNN classifier.
Our accuracy score was much better. 84.4% Train and 78.4% Test. 

We discovered what I believe to be the most important features:
 
9,  57,  65, 106, 120, 129, 200, 206, 242, 286, 337, 339, 379,
85, 443, 454, 473, 476, 494, 49
